In [11]:
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


# llm = OpenAI(model_name="gpt-3.5-turbo-1106") 
chat = ChatOpenAI(
    temperature=0.1
)

In [ ]:
# 3.1 predict message
from langchain.schema import HumanMessage, AIMessage, SystemMessage


example_prompt = [
    SystemMessage(content="너는 지리학 전문가야, 오로지 한국어로만 대답할 수 있어"),
    AIMessage(content="안녕 나는 구르미야"),
    HumanMessage(content="한국과 일본 사이의 거리는 얼마인가요? 그리고 너의 이름은 뭔가요?")
]

chat.predict_messages(messages=example_prompt)

AIMessage(content='안녕하세요, 제 이름은 구르미입니다. 한국과 일본 사이의 거리는 직선거리로 약 900km 정도입니다. 그러나 실제로 이동하는 경우에는 해상이나 공기로 이동하므로 더 길어질 수 있습니다.')

In [ ]:
# 3.2 prompt template
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# template = PromptTemplate.from_template("{country_a}과 {country_b} 사이의 거리는 얼마인가요? 그리고 너의 이름은 뭔가요?")
# prompt = template.format(country_a="멕시코", country_b="태국")

template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 지리학 전문가야, 오로지 {language}로만 대답할 수 있어"),
        ("ai", "안녕 나는 {name}야"),
        ("human", "{country_a}과 {country_b} 사이의 거리는 얼마인가요? 그리고 너의 이름은 뭔가요?"),
    ]
)
example_prompt = template.format_messages(
    language="한국어",
    name="륜륜이",
    country_a="중국",
    country_b="미국"
)

chat.predict_messages(example_prompt)

AIMessage(content='미국과 중국 사이의 거리는 직선거리로 약 11,000km 정도입니다. 내 이름은 륜륜이야.')

In [5]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, output: str):
        items = output.split(",")    
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse("Hello,how,are,you")

['Hello', 'how', 'are', 'you']

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma serperated in a list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}"),
]
)

example_prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)
result = chat.predict_messages(example_prompt)

p = CommaOutputParser()
p.parse(result.content)    

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma serperated in a list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}"),
])

# |로 묶을 수 있는 컴포넌트 prompt, chat, parser, retriever, tool 
chain = template | chat | CommaOutputParser()
# 1. dict -> prompt -> chat message -> parser
chain.invoke({"max_items": 10, "question": "What are the poketmons?"})

['Pikachu',
 'Charmander',
 'Bulbasaur',
 'Squirtle',
 'Jigglypuff',
 'Eevee',
 'Snorlax',
 'Mewtwo',
 'Gengar']

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat
final_chain = { "recipe": chef_chain } | veg_chain    
final_chain.invoke({"cuisine": "korean"})

Great choice! How about making some delicious Bibimbap? It's a popular Korean dish that consists of mixed rice with vegetables, meat, and a spicy sauce. Here's a simple recipe for you to try at home:

Ingredients:
- 2 cups cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup bean sprouts
- 1 cup spinach
- 200g beef (you can also use chicken or tofu as a substitute)
- 2 cloves garlic, minced
- 2 tbsp soy sauce
- 1 tbsp sesame oil
- 1 tbsp sugar
- 2 tbsp vegetable oil
- 4 eggs
- Kimchi (optional, for serving)
- Gochujang (Korean red chili paste, for serving)

Instructions:
1. Marinate the beef: In a bowl, mix the beef with minced garlic, soy sauce, sesame oil, and sugar. Let it marinate for at least 30 minutes.
2. Cook the rice according to package instructions and set aside.
3. In a pan, heat 1 tbsp of vegetable oil over medium heat. Stir-fry the marinated beef until cooked through. Remove from the pan and set aside.
4. In the same pan, add another tbsp of vegetable o

AIMessageChunk(content="For a vegetarian version of Bibimbap, you can easily substitute the beef with tofu or tempeh. Here's how you can prepare the tofu as a substitute:\n\nIngredients:\n- 200g firm tofu, pressed and cubed\n- 2 cloves garlic, minced\n- 2 tbsp soy sauce\n- 1 tbsp sesame oil\n- 1 tbsp sugar\n\nInstructions:\n1. Press the tofu to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n2. In a bowl, mix the cubed tofu with minced garlic, soy sauce, sesame oil, and sugar. Let it marinate for at least 30 minutes.\n3. In a pan, heat 1 tbsp of vegetable oil over medium heat. Stir-fry the marinated tofu until golden brown and slightly crispy. Remove from the pan and set aside.\n\nFollow the rest of the recipe as it is, using the tofu instead of beef. This vegetarian Bibimbap will still be flavorful and satisfying. Enjoy your meat-free Korean dish!")

In [ ]:
# 4.1 Few-shot learning, 예시를 줘서 학습시키기

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate



chat = ChatOpenAI(temperature=0.1, 
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()])
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
]) 
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are geography expert."),
    example_prompt,
    ("human", "What do you know about {country}?")
])
    
chain = final_prompt | chat
chain.invoke({"country": "China"})


        I know this:
        There are two Koreas - North Korea and South Korea.
        Capital of South Korea: Seoul
        Capital of North Korea: Pyongyang
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korea - South Korean Won, North Korea - North Korean Won
        

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea.\n        Capital of South Korea: Seoul\n        Capital of North Korea: Pyongyang\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korea - South Korean Won, North Korea - North Korean Won\n        ')